In [1]:

import pandas as pd 
import numpy as np
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")
import tqdm

# Análise e manipulação de dados

import numpy as np
import pandas as pd
import numpy as np
# Dados séries temporais

from datetime import datetime, timedelta
import pytz
from time import sleep

# Bibliotecas gráficas

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Integração com MetaTrader 5
import datetime as dt
import MetaTrader5 as mt5
import warnings
warnings.filterwarnings("ignore")

import pandasql as ps

In [2]:
mt5.initialize()

True

In [3]:
def baixar_dados(ticker):
    cotacoes_mt5_all = []

    timezone = pytz.timezone("America/Sao_Paulo")
    data_inicio = datetime.now() - timedelta(days=2000)
    data_fim = datetime.now()

    cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_D1, data_inicio, data_fim)
    cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
    cotacoes_mt5_ticker['ticker'] = str(ticker)
    cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
    cotacoes_mt5_all.append(cotacoes_mt5_ticker)
    return pd.concat(cotacoes_mt5_all, axis=0)

In [4]:
def converter_timestamp_em_datetime(timestamp):
        return datetime.utcfromtimestamp(timestamp)

def create_perc(df_ativo, percentual):

    #Ajustar variáveis
    df_ativo['fechAnt'] = df_ativo['close'].shift(1)
    df_ativo['entrada'] = df_ativo['fechAnt'] + (df_ativo['fechAnt'] * percentual)
    df_ativo['percentual'] = percentual

    df_ativo['data'] = df_ativo['time'].apply(converter_timestamp_em_datetime)
    return df_ativo

In [5]:
def create_stats (df_ativo, dias):
    df_ativo['tem_entrada'] = np.where(df_ativo['high'] >= df_ativo['entrada'],1,0)

    df_ativo['alvo'] = np.where(df_ativo['tem_entrada'] == 1 , ((df_ativo['close'] / df_ativo['entrada'] -1) *100)*-1, 0) 
    df_ativo['alvo'] =  np.where(df_ativo['open'] > df_ativo['entrada'], ((df_ativo['close'] / df_ativo['open'] - 1) * 100)*-1, df_ativo['alvo'] )

    df_filtrado = df_ativo.loc[df_ativo['tem_entrada']==1]
    df_filtrado = df_filtrado.tail(dias)
    df_filtrado['alvo_acumulado'] = df_filtrado['alvo'].cumsum()
    return df_filtrado

In [6]:
def calcular_drawdown(df_filtrado):
    df_filtrado['max_acumulado'] = df_filtrado['alvo_acumulado'].cummax()
    df_filtrado['drawdown'] = df_filtrado['alvo_acumulado'] - df_filtrado['max_acumulado']

    # Calcular o drawdown máximo
    drawdown_maximo = df_filtrado['drawdown'].min()

    # Localizar o índice do valor mínimo em 'drawdown'
    indice_drawdown_maximo = df_filtrado['drawdown'].idxmin()


    # Calcular a taxa de acerto
    resultado = df_filtrado['alvo'] > 0
    taxa_acerto = sum(resultado) / len(df_filtrado['alvo'])

    # Separar resultados positivos e negativos
    resultados_positivos = df_filtrado.loc[df_filtrado['alvo'] > 0, 'alvo']
    resultados_negativos = df_filtrado.loc[df_filtrado['alvo'] <= 0, 'alvo']

    try:
        pior_dia = min(resultados_negativos)
        melhor_dia = max(resultados_positivos)
    except:
        pior_dia = 0
        melhor_dia = 0

    
    quantidade_trades = len(resultados_positivos) + len(resultados_negativos)
    # Calcular médias
    media_positivos = resultados_positivos.mean()
    media_negativos = resultados_negativos.mean()

    # Calcular frequência de resultados positivos e negativos
    freq_positivos = len(resultados_positivos) / len(df_filtrado)
    freq_negativos = len(resultados_negativos) / len(df_filtrado)

    # Calcular expectativa matemática
    expectativa = freq_positivos * media_positivos - freq_negativos * abs(media_negativos)

    # Calcular retorno médio diário
    retorno_medio_diario = df_filtrado['alvo'].mean()

    # Calcular desvio padrão diário
    desvio_padrao_diario = df_filtrado['alvo'].std()

    # Calcular índice de Sharpe
    indice_sharpe = np.sqrt(252) * retorno_medio_diario / desvio_padrao_diario
    
    ativo =  df_filtrado.iloc[2]['ticker']
    retorno_acumulado = df_filtrado.iloc[-1]['alvo_acumulado']


    indice_pior_dia = df_filtrado['alvo'].idxmin()
    media_geral = df_filtrado['alvo'].mean()

    data_pior_dia = df_filtrado.loc[indice_pior_dia, 'data']
    primeiro_dia = df_filtrado.iloc[1]['data']

    resultado_dict = {
        'ativo': ativo,
        'percentual': df_filtrado.iloc[2]['percentual'] * 100,
        'Retorno Acumulado': retorno_acumulado,
        'Melhor dia': melhor_dia,
        'Pior Dia':pior_dia, 
        'Data Pior dia': data_pior_dia,
        'Pior Dia': pior_dia,
        'Primeiro Dia': primeiro_dia,
        'Quantidade de Trades': quantidade_trades,
        'Taxa de Acerto': taxa_acerto,
        'Media Geral': media_geral,
        'Média Positivos': media_positivos,
        'Média Negativos': media_negativos,
        'Expectativa Matemática': expectativa,
        'Índice de Sharpe': indice_sharpe,
        'Drawdown Máximo': drawdown_maximo
    }

    return resultado_dict


In [7]:
ibov = ['SEQL3',
'AERI3',
'WEST3',
'TRAD3',
'KRSA3',
'ESPA3',
'IFCM3',
'NGRD3',
'VIIA3',
'MEAL3',
'SGPS3',
'BMGB4',
'LPSB3',
'PDTC3',
'SHOW3',
'CVCB3',
'LUPA3',
'MGLU3',
'UCAS3',
'BRIT3',
'MLAS3',
'COGN3',
'TECN3',
'PINE4',
'NINJ3',
'TCSA3',
'MBLY3',
'HBSA3',
'RAIZ4',
'CIEL3',
'HBOR3',
'CMIN3',
'SYNE3',
'PGMN3',
'QUAL3',
'PFRM3',
'ZAMP3',
'FIQE3',
'OSXB3',
'CSED3',
'POMO3',
'ANIM3',
'HAPV3',
'DMVF3',
'MELK3',
'AGXY3',
'ALPK3',
'CBAV3',
'CLSA3',
'OPCT3',
'PDGR3',
'LVTC3',
'JHSF3',
'RDNI3',
'POMO4',
'HBRE3',
'GFSA3',
'CEAB3',
'SEER3',
'TRIS3',
'LJQQ3',
'WIZC3',
'GUAR3',
'ALLD3',
'AVLL3',
'PETZ3',
'USIM5',
'LWSA3',
'USIM3',
'GRND3',
'MTRE3',
'CAML3',
'GMAT3',
'BOAS3',
'EVEN3',
'MRFG3',
'JALL3',
'PTBL3',
'ECOR3',
'CPLE6',
'BPAN4',
'POSI3',
'DEXP4',
'LAVV3',
'ALPA4',
'DXCO3',
'GOLL4',
'CASH3',
'DEXP3',
'ITSA4',
'ITSA3',
'STBP3',
'RAPT3',
'KEPL3',
'ALPA3',
'BRFS3',
'JSLG3',
'MATD3',
'BEEF3',
'CXSE3',
'RANI3',
'SIMH3',
'PLPL3',
'SOMA3',
'AESB3',
'MEGA3',
'MDNE3',
'ONCO3',
'BMEB4',
'SOJA3',
'CRFB3',
'VITT3',
'VAMO3',
'PRNR3',
'CEDO4',
'MOVI3',
'CMIG4',
'ODPV3',
'RAPT4',
'TTEN3',
'MILS3',
'FHER3',
'ENEV3',
'CCRO3',
'PNVL3',
'SBFG3',
'GOAU3',
'EUCA4',
'BMEB3',
'PORT3',
'TEND3',
'GOAU4',
'FRAS3',
'DASA3',
'ASAI3',
'AURE3',
'MRVE3',
'BMOB3',
'BBDC3',
'TFCO4',
'BRSR3',
'ARML3',
'ENAT3',
'ROMI3',
'BRSR6',
'DESK3',
'TIMS3',
'MYPK3',
'B3SA3',
'ELMD3',
'CSUD3',
'BBDC4',
'TASA4',
'TASA3',
'FLRY3',
'BRBI11',
'VLID3',
'VBBR3',
'AZUL4',
'EUCA3',
'CURY3',
'EMBR3',
'NTCO3',
'CSMG3',
'GGPS3',
'CMIG3',
'UGPA3',
'NEOE3',
'ABCB4',
'LREN3',
'CSAN3',
'JBSS3',
'PCAR3',
'LOGG3',
'VVEO3',
'VULC3',
'SAPR11',
'BLAU3',
'YDUQ3',
'BRAP3',
'DIRR3',
'IGTI11',
'AMBP3',
'RECV3',
'EZTC3',
'RAIL3',
'BRAP4',
'KLBN11',
'BRKM5',
'INTB3',
'SCAR3',
'SMFT3',
'ALSO3',
'TGMA3',
'AALR3',
'ITUB3',
'BRKM3',
'OFSA3',
'CYRE3',
'LAND3',
'TRPL4',
'GGBR3',
'AGRO3',
'SQIA3',
'MULT3',
'PSSA3',
'RADL3',
'ITUB4',
'GGBR4',
'TUPY3',
'ALUP11',
'TOTS3',
'PETR4',
'VIVA3',
'APER3',
'BBSE3',
'EQTL3',
'PETR3',
'BPAC11',
'SMTO3',
'TRPL3',
'RDOR3',
'CPFE3',
'ELET3',
'TAEE11',
'RRRP3',
'ORVR3',
'IRBR3',
'WEGE3',
'ELET6',
'MDIA3',
'EGIE3',
'HYPE3',
'SLCE3',
'LEVE3',
'BBAS3',
'PRIO3',
'LOGN3',
'ENGI11',
'SUZB3',
'FESA4',
'SBSP3',
'CLSC3',
'CLSC4',
'RENT3',
'VALE3',
'ARZZ3']


In [8]:

lista_perc = [0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04]
lista_resultados = []
for ativo in ibov:
    print( f'Baixando os dados de {ativo}')
    for perc in lista_perc:
        try:
            df_ativo = baixar_dados(ativo)
            df_ativo = create_perc(df_ativo, perc)
            df_ativo = create_stats(df_ativo, 30)
            resultado = calcular_drawdown(df_ativo)
            lista_resultados.append(resultado)
        except:
            continue

df_resultado = pd.DataFrame(lista_resultados)

Baixando os dados de SEQL3
Baixando os dados de AERI3
Baixando os dados de WEST3
Baixando os dados de TRAD3
Baixando os dados de KRSA3
Baixando os dados de ESPA3
Baixando os dados de IFCM3
Baixando os dados de NGRD3
Baixando os dados de VIIA3
Baixando os dados de MEAL3
Baixando os dados de SGPS3
Baixando os dados de BMGB4
Baixando os dados de LPSB3
Baixando os dados de PDTC3
Baixando os dados de SHOW3
Baixando os dados de CVCB3
Baixando os dados de LUPA3
Baixando os dados de MGLU3
Baixando os dados de UCAS3
Baixando os dados de BRIT3
Baixando os dados de MLAS3
Baixando os dados de COGN3
Baixando os dados de TECN3
Baixando os dados de PINE4
Baixando os dados de NINJ3
Baixando os dados de TCSA3
Baixando os dados de MBLY3
Baixando os dados de HBSA3
Baixando os dados de RAIZ4
Baixando os dados de CIEL3
Baixando os dados de HBOR3
Baixando os dados de CMIN3
Baixando os dados de SYNE3
Baixando os dados de PGMN3
Baixando os dados de QUAL3
Baixando os dados de PFRM3
Baixando os dados de ZAMP3
B

In [9]:
selecionados = df_resultado.loc[(df_resultado['Taxa de Acerto'] >= 0.75) 
                    & (df_resultado['Pior Dia'] > -1.5) 
                    ].sort_values(['Pior Dia'], ascending=False)
selecionados

,ativo,percentual,Retorno Acumulado,Melhor dia,Pior Dia,Data Pior dia,Primeiro Dia,Quantidade de Trades,Taxa de Acerto,Media Geral,Média Positivos,Média Negativos,Expectativa Matemática,Índice de Sharpe,Drawdown Máximo
933,AURE3,2.0,27.167107,3.317481,-0.586634,2023-11-03,2023-01-25,30,0.800000,0.905570,1.205802,-0.295356,0.905570,14.810245,-0.730208
972,BRSR3,4.0,59.165651,4.762778,-0.791758,2023-11-03,2021-06-14,30,0.866667,1.972188,2.337379,-0.401550,1.972188,20.519811,-0.791758
971,BRSR3,3.5,37.755599,3.847049,-1.278674,2023-11-03,2022-05-16,30,0.800000,1.258520,1.739987,-0.667349,1.258520,13.397487,-1.723502
1543,TAEE11,2.5,11.857129,2.289161,-1.437375,2022-03-04,2021-11-03,30,0.800000,0.395238,0.690822,-0.787100,0.395238,7.320172,-1.437375
